In [1]:
import pandas as pd

In [2]:
model = pd.read_parquet('cleaned_names.parquet')
model.head()

,Name,School,Power
ID,,,
515075,Tiago ASAKAWA,UMBC,15.96
413265,Niklas Weigelt,UMBC,8.38
394588,Ben Beldner,Binghamton,22.71
471360,Justin Meyn,Binghamton,10.06
395308,Joshua Franco,New Jersey Institute of Technology,12.40


In [3]:
df = pd.read_parquet('corrected_HS_times_cleaned.parquet')

In [4]:
df.head()

,ID,50 Y Free Time,50 S Free Time,50 L Free Time,100 Y Free Time,100 S Free Time,100 L Free Time,200 Y Free Time,200 S Free Time,200 L Free Time,...,200 S Fly Points,200 L Fly Points,100 Y IM Points,100 S IM Points,200 Y IM Points,200 S IM Points,200 L IM Points,400 Y IM Points,400 S IM Points,400 L IM Points
0,880302,25.62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,47.61,NaN,NaN,NaN,NaN,NaN
1,1653662,<NA>,<NA>,26.50,<NA>,<NA>,56.48,1:48.93,<NA>,<NA>,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
2,1653662,<NA>,<NA>,26.50,<NA>,<NA>,56.48,1:48.93,<NA>,<NA>,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
3,1131128,22.77,<NA>,<NA>,49.20,<NA>,<NA>,1:48.10,<NA>,<NA>,...,NaN,NaN,100.00,NaN,38.46,NaN,NaN,83.96,NaN,NaN
4,483789,22.71,<NA>,<NA>,51.62,<NA>,<NA>,<NA>,<NA>,2:45.24,...,NaN,NaN,94.93,NaN,100.00,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.drop_duplicates()
df.shape

(511, 89)

In [6]:
# df1 = pd.read_csv('corrected_matches.csv')
df1.head()

,ID,Name,School
0,880302,Lauren Johnson,Bryn Mawr
1,1653662,Kyle Johnson,"University of North Carolina, Wilmington"
2,1653662,Kyle Johnson,Mary Washington
3,1131128,Max Mitchell,Colorado School of Mines
4,483789,Max Mitchell,Ohio Wesleyan


In [7]:
df1 = df1.drop_duplicates()
# df1.shape

(636, 3)

In [15]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from IPython.display import clear_output
def getPowerIndex(swimmer_ID):
	swimmer_url = 'https://swimcloud.com/swimmer/' + str(swimmer_ID)

	url = requests.get(swimmer_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
	url.encoding = 'utf-8'
	
	if url.status_code == 404:
		raise ValueError(f"The swimmer {swimmer_ID} was not found")

	soup = bs(url.text, 'html.parser')
	

	data_array = soup.find_all('li', {'class' : 'c-list-bar__item'}) # returns the container items containing headers for recruiting stats
	for d in data_array:
		if d.find(class_='c-list-bar__subheader').get('title') == 'Power index':
				return float(d.find(class_='c-list-bar__description').text.strip())
	if soup.find('h1', {'class' : 'c-toolbar__title'}).text.strip() == None:
		print("ERROR PROCESSING")
		time.sleep(30)
		return -1.0

	swimmer_name = soup.find('h1', {'class' : 'c-toolbar__title'}).text.strip()

	swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name=' + swimmer_name.replace(' ', '+')

	name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})

	name_url.encoding = 'utf-8'

	name_soup = bs(name_url.text, 'html.parser')
	if name_soup.find('tbody').find_all('tr') == None:
		time.sleep(30)
		return -1.0

	swimmer_list = name_soup.find('tbody').find_all('tr')
	i=0
	
	for swimmer in swimmer_list:
		#check if this is the correct swimmer by looking to see if the link matches their id
		name = swimmer.find(class_='u-text-semi')
		swimmer_link = name['href'] if name else ''
		if(swimmer_link == "/swimmer/" + str(swimmer_ID)):
			#a power index was found for the specified swimmer_ID!
			idx = float(name_soup.find_all('td', {'class' : 'u-text-end'})[i].text.strip())
			print( idx )
			return idx
		i = i + 1

	return -1 #if no swimmer is found with the correct swimmer ID #, return -1

In [16]:
# df1['Power']=-1.0

In [17]:
df1.head()

,ID,Name,School,Power
0,880302,Lauren Johnson,Bryn Mawr,50.15
1,1653662,Kyle Johnson,"University of North Carolina, Wilmington",39.19
2,1653662,Kyle Johnson,Mary Washington,39.19
3,1131128,Max Mitchell,Colorado School of Mines,-1.00
4,483789,Max Mitchell,Ohio Wesleyan,52.63


In [11]:
df1 = df1.reset_index().drop('index',axis=1)

In [12]:
df1.index

RangeIndex(start=0, stop=636, step=1)

In [28]:
for idx, row in df1.iterrows():
    if (df1.at[idx,'Power'] == -1.0) and idx > 391:
        print(row['ID'])
        try:
            power_index = getPowerIndex(row['ID'])
            df1.at[idx, 'Power'] = power_index
        except Exception as e:
            print(f"Error processing ID {row['ID']}: {e}")
        finally:
            if idx % 10 == 0:
                print(f"Processed {idx+1}/{len(df1)} swimmers")
            if idx % 90 == 0:
                # time.sleep(30)
                clear_output(wait=True)

480160
1176801
241151
1155858
611515
38.95
Processed 401/636 swimmers
857206
857206
1720619
984996
234433
1261137
505606
1157016
1826414
Processed 431/636 swimmers
627741
1302736
1562280
403167
626202
1340851
672292
672292
Processed 441/636 swimmers
563453
1428833
2165269
982790
1244294
481898
1193028
Processed 461/636 swimmers
2508192
969605
Processed 471/636 swimmers
960572
1538362
1124256
282752
1486151
757347
2059558
273040
273040
914866
1524052
1399436
451121
1375209
680922
2086360
1441764
Processed 511/636 swimmers
870936
405178
2071993
1184549
922478
Processed 521/636 swimmers
419514
1487318
1445404
401372
1129228
1129228
1182664
2894607
358132
2111399
2336160
Processed 561/636 swimmers
547583
454155
262846
1561152
1180003
734610
2060200
Processed 581/636 swimmers
215568
530353
Processed 591/636 swimmers
1090657
1090657
991070
403111
2663727
2663727
1742522
1030152
Processed 611/636 swimmers
241007
856176
856176
1529304
504944
932597
375141


KeyboardInterrupt: 

In [25]:
df1.Power.value_counts()

Power
-1.00      213
 100.00     48
 39.19       3
 22.62       2
 17.69       2
          ... 
 40.96       1
 86.06       1
 14.37       1
 58.97       1
 74.30       1
Name: count, Length: 359, dtype: int64

In [32]:
df1[df1.Name=='Nicholas Edwards']

,ID,Name,School,Power
299,663038,Nicholas Edwards,Rose-Hulman Institute of Technology,25.38


In [31]:
df1.at[299, 'Power'] = 25.38

In [38]:
# Identify rows in model that are also present in df1 based on 'Name' and 'School'
model_to_drop = model.merge(df1[['Name', 'School']], on=['Name', 'School'], how='inner')

# Filter out the rows from model that are in model_to_drop
model_filtered = model[~model.set_index(['Name', 'School']).index.isin(model_to_drop.set_index(['Name', 'School']).index)]

# Reset the index if needed
model_filtered.reset_index(drop=True, inplace=True)

# Display the filtered DataFrame
print(model_filtered.shape)


(12156, 3)


In [35]:
df_filtered.shape

(12698, 3)

In [39]:
model_filtered[model_filtered.Name=='Nicholas Edwards']

,Name,School,Power
3419,Nicholas Edwards,Western Connecticut State University,100.0


In [40]:
df1 = df1[df1['Power'] != -1.0]

In [41]:
# We have completed len(idxs)
df1.shape

(425, 4)

In [ ]:
df1 = df1.drop_duplicates()
df1.shape

In [44]:
model = model.reset_index()
model.head()

,ID,Name,School,Power
0,515075,Tiago ASAKAWA,UMBC,15.96
1,413265,Niklas Weigelt,UMBC,8.38
2,394588,Ben Beldner,Binghamton,22.71
3,471360,Justin Meyn,Binghamton,10.06
4,395308,Joshua Franco,New Jersey Institute of Technology,12.40


In [ ]:
model[model.ID==

In [45]:
df1.head()

,ID,Name,School,Power
0,880302,Lauren Johnson,Bryn Mawr,50.15
1,1653662,Kyle Johnson,"University of North Carolina, Wilmington",39.19
2,1653662,Kyle Johnson,Mary Washington,39.19
4,483789,Max Mitchell,Ohio Wesleyan,52.63
5,403694,Lauren Gray,University of Arkansas-Little Rock,34.05


In [46]:
comb = pd.concat([model,df1])

In [47]:
comb.head()

,ID,Name,School,Power
0,515075,Tiago ASAKAWA,UMBC,15.96
1,413265,Niklas Weigelt,UMBC,8.38
2,394588,Ben Beldner,Binghamton,22.71
3,471360,Justin Meyn,Binghamton,10.06
4,395308,Joshua Franco,New Jersey Institute of Technology,12.40


In [48]:
comb.shape

(13123, 4)

In [49]:
comb = comb.drop_duplicates()
comb.shape

(13123, 4)

In [50]:
comb.dtypes

ID         object
Name       object
School     object
Power     float64
dtype: object

In [54]:
comb.ID = comb.ID.astype('int64')
comb.Name = comb.Name.astype('category')
comb.School = comb.School.astype('category')

In [53]:
comb.at[7820, 'ID'] = 368904

In [55]:
comb.dtypes

ID           int64
Name      category
School    category
Power      float64
dtype: object

In [56]:
comb.to_parquet('corrected_cleaned_names.parquet',index=False)